**IMPORTANT** 

- For requirements and initial setup go to https://github.com/OliveiraEdu/OpenScience/Readme.md;
- To execute the notebook run all cells.

# Notebook 3 - Open Science Platform - Project Account Creation

## Activities

1 - Deploys a smart contract into the Iroha 1 blockchain for account creation;

2 - Creates a project account into the blockchain;

3 - Queries the blockchain to confirm the succesful creation of the account;

4 - Deploys a smart contract into the Iroha 1 blockchain for details (attributes) setting;

5 - Sets details for the project account;

6 - Queries the blockchain again to confirm the succesfull details setting.

Every account succesfully created is dumped to `datasets/projects.json` for later use and evidence collection.  

In [10]:
# from name_generator import left, right
from utilities import *
from project_id_generator import * # - Random generated synthetic project ID
from iroha_helper import *
from loguru import logger
from dump_to_json import ProjectAccount, update_or_append_project_metadata
from ipfs_functions import download_json_from_ipfs
from synthetic_project_json_ld import *
from new_helper import extract_account_metadata_cid_from_data, process_raw_data, load_variable
from loguru import logger

In [11]:
DOMAIN = "test"

1 - Loads the smart contract hash for account creation into the Iroha 1 blockchain.  

In [12]:
# Loading the variable from JSON
create_account_contract_hash = load_variable("Create Account Hash")
logger.info(create_account_contract_hash)


2025-02-09 22:16:52,741 - INFO - Variable 'Create Account Hash' successfully loaded from temp/Create Account Hash.json
2025-02-09 22:16:52.742 | INFO     | __main__:<module>:3 - b'a76bb4a33334ef018daf965042478cfc41229121be8e4d461a3dc9f9c709c2ce'


2 - Creates a project account into the blockchain;

In [13]:
project_private_key = IrohaCrypto.private_key()
project_public_key = IrohaCrypto.derive_public_key(project_private_key).decode("utf-8")

project_id = generate_project_id()

logger.info(f"Creating project with name: {project_id}")

# - Create account
create_account_contract_address = integration_helpers.get_engine_receipts_address(create_account_contract_hash)
create_project_account(create_account_contract_address, project_id, DOMAIN, project_public_key)


2025-02-09 22:16:52.801 | INFO     | __main__:<module>:6 - Creating project with name: 38273
2025-02-09 22:16:52.803 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_address"
2025-02-09 22:16:52.824 | INFO     | integration_helpers:get_engine_receipts_address:96 - Retrieved contract address for transaction hash: b'a76bb4a33334ef018daf965042478cfc41229121be8e4d461a3dc9f9c709c2ce'
2025-02-09 22:16:52.825 | DEBUG    | integration_helpers:get_engine_receipts_address:97 - Contract address: 93F622673AA712B7298FDD2212E3228D8D9DD8BF
2025-02-09 22:16:52.827 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_engine_receipts_address"
2025-02-09 22:16:52.828 | DEBUG    | integration_helpers:tracer:35 - 	Entering "create_project_account"
2025-02-09 22:16:52,844 - INFO - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-02-09 22:16:52,845 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-02-09 22:16:55,510 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-02-09 22:16

3 - Queries the blockchain to confirm the succesful creation of the account;

In [14]:
#Query - GetAccountDetail

# - Get account info
project_account_hash = get_account(create_account_contract_address, project_id, DOMAIN)
address = integration_helpers.get_engine_receipts_result(project_account_hash)


2025-02-09 22:16:55.548 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_account"
2025-02-09 22:16:55,551 - WARNING - create_account_contract_address: 93F622673AA712B7298FDD2212E3228D8D9DD8BF
2025-02-09 22:16:55,569 - INFO - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-02-09 22:16:55,571 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-02-09 22:16:58,552 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-02-09 22:16:58,560 - INFO - ('COMMITTED', 5, 0)
2025-02-09 22:16:58,563 - WARNING - Account Data: {'address': '93F622673AA712B7298FDD2212E3228D8D9DD8BF', 'hex_hash': '47649fdd030340125533a609c5bb8cf023ecede6ae9fbe3d54520d78af8b1c93', 'timestamp': 1739139418563}
2025-02-09 22:16:58.566 | DEBUG    | integration_helpers:tracer:35 - 	Entering "append_to_json_file"
2025-02-09 22:16:58,569 - INFO - Appended data under '38273@test' in logs/account_data.json
2025-02-09 22:16:58.570 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "append_to_json_file"
2025-02-09 22:16:58.571

4 - Deploys a smart contract into the Iroha 1 blockchain for details (attributes) setting;

In [15]:
# Loading the variable from JSON
account_detail_contract_hash = load_variable("Account Detail Contract Hash")
logger.info(account_detail_contract_hash)

2025-02-09 22:16:58,625 - INFO - Variable 'Account Detail Contract Hash' successfully loaded from temp/Account Detail Contract Hash.json
2025-02-09 22:16:58.627 | INFO     | __main__:<module>:3 - b'86452d50ed5d8cdb132dc84c548608c8454a8a993a6deab3384e17e1aee55ec0'


5 - Sets details for the project account:

In [16]:
# Generate synthetic project metadata JSON (assumed to be done already)
project_metadata_json = generate_synthetic_entry_ld()
account_metadata_cid = upload_json_to_ipfs(project_metadata_json)  # Assuming this is done before calling the function

# Call the function to update or append a new entry to the JSON-LD file
updated_entry_count = update_or_append_project_metadata(
    project_id, 
    project_metadata_json, 
    account_metadata_cid
)


address = integration_helpers.get_engine_receipts_address(account_detail_contract_hash)

# Setting account detail using the generated project metadata CID
hash = set_account_detail(
    address, 
    f"{project_id}@{DOMAIN}",  # Project ID as account
    "account_metadata_cid",    # The key we're setting
    account_metadata_cid       # The value (CID from IPFS)
)

2025-02-09 22:16:58.728 | INFO     | dump_to_json:update_or_append_project_metadata:216 - Updated existing project entry for project ID: 38273@test
2025-02-09 22:16:58.731 | INFO     | dump_to_json:update_or_append_project_metadata:235 - Current total entries in 'datasets/projects.json': 2
2025-02-09 22:16:58.732 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_address"


2025-02-09 22:16:58.752 | INFO     | integration_helpers:get_engine_receipts_address:96 - Retrieved contract address for transaction hash: b'86452d50ed5d8cdb132dc84c548608c8454a8a993a6deab3384e17e1aee55ec0'
2025-02-09 22:16:58.754 | DEBUG    | integration_helpers:get_engine_receipts_address:97 - Contract address: A518E3EE4D45C20EF834361093305D2500A46228
2025-02-09 22:16:58.755 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_engine_receipts_address"
2025-02-09 22:16:58.756 | DEBUG    | integration_helpers:tracer:35 - 	Entering "set_account_detail"
2025-02-09 22:16:58,772 - INFO - None
2025-02-09 22:16:58,774 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-02-09 22:17:01,592 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-02-09 22:17:01,600 - INFO - ('COMMITTED', 5, 0)
2025-02-09 22:17:01.602 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_result"
2025-02-09 22:17:01.625 | INFO     | integration_helpers:get_engine_receipts_result:139 - Resul

6 - Queries the blockchain to confirm the succesful details setting

In [17]:
# - Get account info
get_account_hash = get_account(create_account_contract_address, project_id, DOMAIN)
details = integration_helpers.get_engine_receipts_result(get_account_hash)

# Process the raw data
account_metadata_cid, project_id = process_raw_data(details) #TO DO - From now

2025-02-09 22:17:01.648 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_account"
2025-02-09 22:17:01,652 - WARNING - create_account_contract_address: 93F622673AA712B7298FDD2212E3228D8D9DD8BF
2025-02-09 22:17:01,669 - INFO - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-02-09 22:17:01,670 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-02-09 22:17:04,629 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-02-09 22:17:04,639 - INFO - ('COMMITTED', 5, 0)
2025-02-09 22:17:04,641 - WARNING - Account Data: {'address': '93F622673AA712B7298FDD2212E3228D8D9DD8BF', 'hex_hash': 'fef1b12359f9df450c89c0a85f37fe3a619b75c181e97ed9ba09251cc0af2b77', 'timestamp': 1739139424641}
2025-02-09 22:17:04.643 | DEBUG    | integration_helpers:tracer:35 - 	Entering "append_to_json_file"
2025-02-09 22:17:04,646 - INFO - Appended data under '38273@test' in logs/account_data.json
2025-02-09 22:17:04.647 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "append_to_json_file"
2025-02-09 22:17:04.649

7 - Retrieves Project metadata (JSON-LD formatted) from IPFS

In [18]:
project_metadata = download_json_from_ipfs(account_metadata_cid)
logger.info(f"User Metadata: {project_metadata}")

2025-02-09 22:17:04.715 | INFO     | __main__:<module>:2 - User Metadata: {'@context': {'dc': 'http://purl.org/dc/terms/', 'schema': 'http://schema.org/'}, '@type': 'schema:ResearchProject', 'dc:abstract': 'This study aims to investigate the effects of electric vehicles on manufacturing industry and propose strategies for improvement.', 'schema:endDate': '2025-10-06', 'schema:funding': {'@type': 'schema:Organization', 'schema:name': 'European Union Horizon 2020'}, 'schema:keywords': ['electric vehicles', 'manufacturing industry', 'resource optimization'], 'schema:location': {'@type': 'schema:Place', 'schema:name': 'Accra, Ghana'}, 'schema:name': 'Exploring the Effects of electric vehicles on manufacturing industry', 'schema:startDate': '2023-10-09'}
